In [1]:
!pip install dataretrieval
from dataretrieval import nwis
import numpy as np
import pandas as pd
from datetime import datetime
from google.colab import drive # Mount Google Drive
drive.mount('/content/drive')
current_time = datetime.now()
print(current_time.strftime("%Y-%m-%d %H:%M:%S"))

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
2025-08-13 14:36:31


In [2]:
kmeans_ws_path = '/content/drive/MyDrive/kmeans_ws.csv'
#ws_char_path  = '/content/drive/MyDrive/ws_charateristics_202506191550.csv'
site_amp3_path  = '/content/drive/MyDrive/site_amp3_202506191536.csv'

kmeans_ws_df  = pd.read_csv(kmeans_ws_path)
#ws_char_df    = pd.read_csv(ws_char_path)
site_amp3_df  = pd.read_csv(site_amp3_path)

# 3️⃣ (Optional) sanity‑check
print(kmeans_ws_df.head())
#print(ws_char_df.head())
print(site_amp3_df.head())

merged_df = pd.merge(
    kmeans_ws_df[['site_no', 'labels']],
#    ws_char_df[['site_no', 'dam_and_diversion']],
    site_amp3_df[['site_no','dam_and_diversion','amp'  ,'ph_l' ]],
    on='site_no',
    how='inner'  # or 'left', 'right', 'outer' depending on your needs
)

print(merged_df)

df_summer = pd.read_csv("/content/drive/MyDrive/df_cleaned_summer.csv")

# Merge with the existing dataframe
reg_df = pd.merge(merged_df, df_summer, on='site_no')

reg_df

    site_no  drain_area_va   alt_va                        labels  \
0  12305000        11740.0  1794.75   Low-Elevation Large Streams   
1  12322000        13700.0  1700.00   Low-Elevation Large Streams   
2  13092747          259.0  3630.00  Mid-Elevation Medium Streams   
3  13150430           70.0  4834.45  Mid-Elevation Medium Streams   
4  13154500        35800.0  2495.13   Low-Elevation Large Streams   

   drain_area_km2        alt_m  
0    30406.309266   547.039800  
1    35482.660728   518.160000  
2      670.803586  1106.424000  
3      181.298266  1473.540360  
4    92721.113434   760.515624  
   Unnamed: 0      site       amp       ph_l   site_no  dec_lat_va  \
0           0  12305000  0.501713  28.067854  12305000   48.616883   
1           1  12322000  0.595400  22.359741  12322000   48.996389   
2           2  13092747  0.471255   6.327435  13092747   42.562500   
3           3  13150430  0.632628  -7.784095  13150430   43.323361   
4           4  13154500  0.439203   2

,site_no,labels,dam_and_diversion,amp,ph_l,tmean,Date,rad_d,Days_Since_October_1st,00010_Mean,00060_Mean,Month,Year
0,12305000,Low-Elevation Large Streams,Dam,0.501713,28.067854,15.100000,2012-06-01,4.183052,243,8.5,31500.0,6,2012
1,12305000,Low-Elevation Large Streams,Dam,0.501713,28.067854,13.240001,2012-06-02,4.200266,244,8.5,37500.0,6,2012
2,12305000,Low-Elevation Large Streams,Dam,0.501713,28.067854,14.009001,2012-06-03,4.217481,245,8.5,40100.0,6,2012
3,12305000,Low-Elevation Large Streams,Dam,0.501713,28.067854,16.588001,2012-06-04,4.234695,246,9.5,39900.0,6,2012
4,12305000,Low-Elevation Large Streams,Dam,0.501713,28.067854,10.858001,2012-06-05,4.251909,247,10.0,42200.0,6,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...
28485,13342500,Low-Elevation Large Streams,Dam and diversion,0.460745,9.294641,17.330099,2024-08-27,5.680688,330,11.4,10500.0,8,2024
28486,13342500,Low-Elevation Large Streams,Dam and diversion,0.460745,9.294641,15.716600,2024-08-28,5.697902,331,10.9,10600.0,8,2024
28487,13342500,Low-Elevation Large Streams,Dam and diversion,0.460745,9.294641,18.979599,2024-08-29,5.715116,332,11.1,10600.0,8,2024
28488,13342500,Low-Elevation Large Streams,Dam and diversion,0.460745,9.294641,22.762400,2024-08-30,5.732331,333,11.3,10500.0,8,2024


In [3]:
#ordered_sites = reg_df.sort_values('dam_and_diversion')['site_no'].unique()

#print("ordered_sites",ordered_sites)

In [4]:




# Convert the 'dam_and_diversion' column to a Categorical type with the specified order
reg_df['dam_and_diversion'] = pd.Categorical( reg_df['dam_and_diversion'], categories= ["No flow regulation", "Diversion", "Dam", "Dam and diversion"], ordered=True)

label_order = ["High-Elevation Small Streams", "Mid-Elevation Medium Streams", "Low-Elevation Large Streams", "Very Large Streams"]

reg_df['labels'] = pd.Categorical( reg_df['labels'], categories= label_order, ordered=True)


ordered_sites = reg_df.sort_values( by = ['dam_and_diversion','labels' ])['site_no'].unique()

In [5]:
#print(df.head())
print("ordered_sites:",ordered_sites)
#print(ax1.get_xticklabels())
#print(type(ordered_sites))
#print( (ordered_sites.shape))
arr = np.arange(1,  ordered_sites.shape[0]+1)
print("arr:",arr)

ordered_sites: [13296000 13310800 13310850 13311000 13311250 13311450 13176400 13340000
 13340600 13304050 13092747 13150430 13307000 13211205 13213000 13192200
 12305000 12322000 13341050 13190500 13154500 13342500 13317660]
arr: [ 1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23]


In [6]:
# Create the DataFrame
ordered_sites_label = pd.DataFrame({
    'ordered_sites': ordered_sites,
    'arr': arr
})

# Display the DataFrame
print(ordered_sites_label)


    ordered_sites  arr
0        13296000    1
1        13310800    2
2        13310850    3
3        13311000    4
4        13311250    5
5        13311450    6
6        13176400    7
7        13340000    8
8        13340600    9
9        13304050   10
10       13092747   11
11       13150430   12
12       13307000   13
13       13211205   14
14       13213000   15
15       13192200   16
16       12305000   17
17       12322000   18
18       13341050   19
19       13190500   20
20       13154500   21
21       13342500   22
22       13317660   23


get lat and lon

In [7]:



# Create empty lists to store the fetched latitude and longitude
latitudes = []
longitudes = []
names=[]

# Iterate over each site ID in the 'ordered_sites' column
for site_id in ordered_sites_label['ordered_sites']:
    try:
        # Use dataretrieval's nwis.get_site_info to get site details
        site_info_tuple = nwis.get_info(sites= str(site_id) )
        site_info= site_info_tuple[0]
        #print( site_info['station_nm'])
        # The result is a DataFrame; we extract the 'dec_lat_va' and 'dec_long_va'
        # Check if the result is not empty before trying to access it
        if not site_info.empty:
            latitude = site_info['dec_lat_va'].iloc[0]
            longitude = site_info['dec_long_va'].iloc[0]
            station_nm= [ site_info['station_nm'].iloc[0] ]
            # Append the fetched latitude and longitude to the respective lists
            latitudes.append(latitude)
            longitudes.append(longitude)
            names.append(station_nm)
            #print(station_nm)

        else:
            # Append NaN if no data is found for the site
            latitudes.append(None)
            longitudes.append(None)
            names.append(None)
            print(f"No data found for site ID: {site_id}")

    except Exception as e:
        # Handle potential errors during the API call
        print(f"Error fetching data for site ID {site_id}: {e}")
        latitudes.append(None)
        longitudes.append(None)
        names.append(None)

# Add the new latitude and longitude lists as columns to the DataFrame
ordered_sites_label['lat'] = latitudes
ordered_sites_label['lon'] = longitudes
ordered_sites_label['names'] = names

# Print the updated DataFrame
print(ordered_sites_label)

    ordered_sites  arr        lat         lon  \
0        13296000    1  44.278889 -114.733889   
1        13310800    2  44.902250 -115.325667   
2        13310850    3  44.889528 -115.360222   
3        13311000    4  44.905722 -115.329500   
4        13311250    5  44.934778 -115.336694   
5        13311450    6  44.936361 -115.337222   
6        13176400    7  42.261369 -116.868442   
7        13340000    8  46.478333 -116.257500   
8        13340600    9  46.840556 -115.621111   
9        13304050   10  44.688792 -113.370406   
10       13092747   11  42.562500 -114.494722   
11       13150430   12  43.323361 -114.108350   
12       13307000   13  45.322500 -114.440000   
13       13211205   14  43.677353 -116.701131   
14       13213000   15  43.781667 -116.972778   
15       13192200   16  43.550556 -115.721944   
16       12305000   17  48.616883 -116.049194   
17       12322000   18  48.996389 -116.507500   
18       13341050   19  46.500278 -116.392500   
19       13190500   

In [8]:
#ordered_sites_label.info()

ordered_sites_label['names'] = ordered_sites_label['names'].astype(str)

ordered_sites_label["names"] = ordered_sites_label["names"].str.replace(", ID", "", regex=False).str.replace(" ID", "", regex=False)

# Iterate through each list in the 'names' column and apply string replacement
#ordered_sites_label["names"] = ordered_sites_label["names"].apply(lambda x: [item.replace("[", "").replace("]", "") for item in x] if isinstance(x, list) else x)

In [9]:
ordered_sites_label

,ordered_sites,arr,lat,lon,names
0,13296000,1,44.278889,-114.733889,['YANKEE FORK SALMON RIVER NR CLAYTON']
1,13310800,2,44.902250,-115.325667,['EFSF SALMON R ABV MEADOW CRK NR STIBNITE']
2,13310850,3,44.889528,-115.360222,['MEADOW CREEK NR STIBNITE']
3,13311000,4,44.905722,-115.329500,['EF OF SF SALMON RIVER AT STIBNITE']
4,13311250,5,44.934778,-115.336694,['EFSF SALMON R ABV SUGAR CRK NR STIBNITE']
5,13311450,6,44.936361,-115.337222,['SUGAR CREEK NR STIBNITE']
6,13176400,7,42.261369,-116.868442,['EF OWYHEE RIVER AT CRUTCHER CROSSING']
7,13340000,8,46.478333,-116.257500,['CLEARWATER RIVER AT OROFINO']
8,13340600,9,46.840556,-115.621111,['NF CLEARWATER RIVER NR CANYON RANGER STATION']
9,13304050,10,44.688792,-113.370406,['Big Timber Creek Abv Mouth at Leadore']
